In [ ]:
# --- Run this cell once to install the libraries ---
%pip install selenium webdriver-manager pandas -q
print("✅ Libraries installed successfully!")

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

print("✅ Libraries imported successfully!")

In [ ]:
# --- ⬇️ PASTE THE URL YOU WANT TO SCRAPE HERE ⬇️ ---
target_url = 'https://www.google.com'  # Example: 'https://www.example.com'
# ----------------------------------------------------

# --- Define the name of the output file ---
output_csv_file = 'exported_cookies.csv'
# ------------------------------------------

print(f"Target URL set to: {target_url}")
print(f"Output file will be named: {output_csv_file}")

In [ ]:
# Cell 4 (Updated) - 🚀 Run the Scraper

import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# --- Install a specific version of Chrome and ChromeDriver ---
# This can help resolve compatibility issues
!apt-get update
!apt install chromium-browser chromium-chromedriver
# -----------------------------------------------------------


# --- New: Configure Chrome Options to be more permissive ---
chrome_options = webdriver.ChromeOptions()
# This preference attempts to disable the browser's built-in cookie blocking
prefs = {"profile.cookie_controls_mode": 0}
chrome_options.add_experimental_option("prefs", prefs)
# To run without a visible browser window, uncomment the next line:
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu") # Add this line
chrome_options.add_argument("--no-first-run") # Add this line
chrome_options.add_argument("--no-default-browser-check") # Add this line
chrome_options.add_argument("--window-size=1280,800") # Add this line


driver = None
try:
    print("🚀 Starting the browser with permissive settings...")
    # Install and use a specific version of Chrome if needed
    service = ChromeService(ChromeDriverManager().install())

    driver = webdriver.Chrome(
        service=service,
        options=chrome_options
    )

    print(f"Navigating to {target_url}...")
    driver.get(target_url)

    print("Waiting 7 seconds for page and iframes to load...")
    time.sleep(7)

    # 1. Get first-party cookies from the main document
    all_cookies = driver.get_cookies()
    print(f"✅ Found {len(all_cookies)} first-party cookies.")

    # 2. --- New: Attempt to find and switch to iFrames ---
    print("Scanning for iFrames to find third-party cookies...")
    iframes = driver.find_elements(By.TAG_NAME, 'iframe')
    print(f"Found {len(iframes)} iFrames on the page.")

    for frame in iframes:
        try:
            # Switch context to the iframe
            driver.switch_to.frame(frame)
            # Get cookies set within the iframe's context
            iframe_cookies = driver.get_cookies()
            if iframe_cookies:
                print(f"  > Found {len(iframe_cookies)} cookies in an iframe.")
                all_cookies.extend(iframe_cookies)
            # IMPORTANT: Switch back to the main page's context
            driver.switch_to.default_content()
        except Exception as e:
            print(f"  > Could not switch to an iframe or get cookies: {e}")
            driver.switch_to.default_content() # Ensure we switch back even if there's an error

    if not all_cookies:
        print("❌ No cookies were found on this page.")
    else:
        # Remove duplicate cookies by converting to a set of tuples and back
        unique_cookies_tuples = {tuple(d.items()) for d in all_cookies}
        unique_cookies = [dict(t) for t in unique_cookies_tuples]

        print(f"\n✅ Found a total of {len(unique_cookies)} unique cookies. Preparing to save...")
        df = pd.DataFrame(unique_cookies)

        # Convert expiry timestamp to readable date if it exists
        if 'expiry' in df.columns:
            df['expiry'] = pd.to_datetime(df['expiry'], unit='s', errors='coerce')

        df.to_csv(output_csv_file, index=False)
        print(f"\n🎉 Success! All found cookies saved to '{output_csv_file}'")

finally:
    if driver:
        print("Closing the browser...")
        driver.quit()

In [ ]:
import os

if os.path.exists(output_csv_file):
    print(f"--- Displaying contents of '{output_csv_file}' ---")
    # Read the CSV file back into a DataFrame and display it
    saved_df = pd.read_csv(output_csv_file)
    display(saved_df.head())
else:
    print(f"'{output_csv_file}' not found. Please run the scraper cell first.")